In [82]:
import pandas as pd
%matplotlib notebook
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file
from bokeh.charts import Bar, output_file, show
import bokeh.plotting as bplt
from bokeh.palettes import Dark2_5 as palette
from bokeh.models import Label, Title
import itertools  
from bokeh.layouts import gridplot,row,column

from IPython.core.display import display, HTML
from bokeh.io import output_notebook
plt.style.use('ggplot')
output_notebook()

Loading BokehJS ...

In [2]:
directory = '/home/georgeha/repos/midas_exps/streaming/k-means/spark/'

def plot_producer(producer_dir,producer_filename,p_number,fig_count,txt):
    
    rates =pd.read_csv(directory + producer_dir + producer_filename)
    x_values = rates['Num_Messages'].tolist()
    y_values = rates['KB/sec'].tolist()
    
    f = plt.figure()
    plt.xlim(0,12500)
    plt.plot(x_values,y_values)
    plt.ylabel('KB/s')
    plt.xlabel('Msg number')
    plt.title("Producer " + str(p_number)+ "  throughput")
    text = 'Figure ' + str(fig_count) + ' : ' + txt
    f.text(.20, .02, text, ha='center')
    
    return (x_values,y_values)

In [122]:
directory = '/home/georgeha/repos/midas_exps/streaming/k-means/spark/'
def plot_scheduling_and_Total_Delay(spark_metrics_dir,fig_count):
    
    metrics = pd.read_csv(directory + spark_metrics_dir ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay'].tolist()    
    TotalDelay = metrics['TotalDelay'].tolist()
    
    scheduling_delay = map(lambda x: x/1000, scheduling_delay)  # convert sec from milisec
    TotalDelay = map(lambda x: x/1000, TotalDelay)

    x_values = range(0,metrics['SchedulingDelay'].count())
    # create a new plot with a title and axis labels
    p = figure(title="Figure " + str(fig_count) + ": Spark Scheduling delay", x_axis_label='miniBatch Number', y_axis_label='Delay in sec')
    # add a line renderer with legend and line thickness
    p.line(x_values, scheduling_delay, legend="Scheduling Delay", line_width=2,line_color="red")
    p.line(x_values,TotalDelay,legend="Total Delay", line_width=2)
    #throughput = (TotalDelay - scheduling_delay)/metrics['NumberRecords']
    bplt.show(p)
    return (x_values,scheduling_delay)

In [70]:
def plot_throughput_per_mini_batch(spark_metrics_dir,fig_count):
    
    metrics = pd.read_csv(directory + spark_metrics_dir ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay']
    TotalDelay = metrics['TotalDelay']
    x_values = range(0,scheduling_delay.count())    
    throughput = metrics['NumberRecords']/60      # check That
    # create a new plot with a title and axis labels
    p = figure(title="Figure " + str(fig_count) + " : Spark throughput/mini batch  - batch=60sec", x_axis_label='miniBatch Number', y_axis_label='records/sec')

    # add a line renderer with legend and line thickness
    p.line(x_values, throughput.tolist(), legend="Throughput", line_width=2,line_color="green")

    # show the results
    bplt.show(p)
    return  (x_values,throughput.tolist())

All the data are from extending the class onBatchCompleted of the Spark Listener

Record: 1 record is one Kafka message. <br>
1 Kafka message is an array of 500 3-dim points

# Producers on 1 wrangler node

In [124]:
directory = directory + '1-node/'

In [46]:
#p_111_1_5k = plot_producer('/1-1-1/producer-1/','stdout-20171003-153435.csv',1,2,'5k-1-1-1')

In [ ]:
p_111_1_5k = plot_producer('/1-1-1/producer-1/','stdout-20171003-153435.csv',1,2,'5k-1-1-1')

## Paragraph 

In [127]:
#a111 = plot_throughput_per_mini_batch('/1-1-1/spark-metrics-20171003-153347.csv',1)
d111 = plot_scheduling_and_Total_Delay('/1-1-1/spark-metrics-20171003-153347.csv',1)

In [116]:
#b111 = pd.read_csv(directory+'1-1-1/spark-metrics-20171003-153347.csv')
#b111

In [40]:
print 'Number of Records Consumed from Spark Consumer: %d ' % b111[' NumberRecords'].sum()

Number of Records Consumed from Spark Consumer: 5655 


In [126]:
#a311 = plot_throughput_per_mini_batch('/3-1-1/spark-metrics-20171004-213423.csv',2)
d311 = plot_scheduling_and_Total_Delay('/3-1-1/spark-metrics-20171004-213423.csv',2)

In [41]:
b311 = pd.read_csv(directory+'/3-1-1/spark-metrics-20171004-213423.csv').head()
b311

In [50]:
print 'Number of Records Consumed from Spark Consumer: %d ' % b311[' NumberRecords'].sum()
print 'The number of producers is tripled but the consumption is increased by x3.4'

Number of Records Consumed from Spark Consumer: 19399 
The number of producers is tripled but the consumption is increased by x3.4


In [128]:
#a611 = plot_throughput_per_mini_batch('/6-1-1/spark-metrics-20171005-001351.csv',3)
d611 = plot_scheduling_and_Total_Delay('/6-1-1/spark-metrics-20171005-001351.csv',3)

In [44]:
b611 = pd.read_csv(directory+'/6-1-1/spark-metrics-20171005-001351.csv').head()
b611

,BatchTime,SubmissionTime,SchedulingDelay,TotalDelay,NumberRecords,Scenario
0,1507180440000 ms,2017-10-05T00:14:03,4,2429,0,1_Producer_Count
1,1507180500000 ms,2017-10-05T00:15:12,1,39005,3493,1_Producer_Count
2,1507180560000 ms,2017-10-05T00:16:30,1,132486,9418,1_Producer_Count
3,1507180620000 ms,2017-10-05T00:17:56,47136,199035,8246,1_Producer_Count
4,1507180680000 ms,2017-10-05T00:19:19,115186,294126,10455,1_Producer_Count


In [49]:
print 'Number of Records Consumed from Spark Consumer: %d ' % b611[' NumberRecords'].sum()
print 'The number of producers is doubled but the consumption is increased by x1.6'

Number of Records Consumed from Spark Consumer: 31612 
The number of producers is doubled but the consumption is increased by x1.6


In [129]:
#a1211 = plot_throughput_per_mini_batch('/12-1-1/spark-metrics-20171005-161813.csv',4)
d1211 = plot_scheduling_and_Total_Delay('/12-1-1/spark-metrics-20171005-161813.csv',4)

In [55]:
b1211 = pd.read_csv(directory+'/12-1-1/spark-metrics-20171005-161813.csv').head()

print 'Number of Records Consumed from Spark Consumer: %d ' % b1211[' NumberRecords'].sum()
print 'The number of producers is doubled but the consumption is increased by 3.8' 

Number of Records Consumed from Spark Consumer: 121360 
The number of producers is doubled but the consumption is increased by 3.8


In [131]:
#a2411 = plot_throughput_per_mini_batch('/24-1-1/spark-metrics-20171005-183434.csv',5)
d2411 = plot_scheduling_and_Total_Delay('/24-1-1/spark-metrics-20171005-183434.csv',5)


In [ ]:
print "Note: This line should not be continuous."

In [60]:
b2411 = pd.read_csv(directory+'/24-1-1/spark-metrics-20171005-183434.csv')
print 'Number of Records Consumed from Spark Consumer: %d ' % b2411[' NumberRecords'].sum()
print 'The number of producers is doubled but the consumption is increased by 1.47' 

Number of Records Consumed from Spark Consumer: 179155 
The number of producers is doubled but the consumption is increased by 1.47


#### Throughput per minibatch on 1 - node

In [92]:
p = figure(title="Figure " + str(6) + ": Spark throuhgput per minibatch", x_axis_label='miniBatch Number', y_axis_label='throughput in Rec/sec')

p.line(a111[0], a111[1], legend="1-1-1", line_width=2,line_color="red")
p.line(a311[0], a311[1], legend="3-1-1", line_width=2,line_color="blue")
p.line(a611[0], a611[1], legend="6-1-1", line_width=2,line_color="green")
p.line(a1211[0], a1211[1], legend="12-1-1", line_width=2,line_color="black")
p.line(a2411[0], a2411[1], legend="24-1-1", line_width=2,line_color="yellow")


bplt.show(p)

In [132]:
p = figure(title="Figure " + str(6) + ": Spark scheduling delay", x_axis_label='miniBatch Number', y_axis_label='throughput in Rec/sec')

p.line(d111[0], d111[1], legend="1-1-1", line_width=2,line_color="red")
p.line(d311[0], d311[1], legend="3-1-1", line_width=2,line_color="blue")
p.line(d611[0], d611[1], legend="6-1-1", line_width=2,line_color="green")
p.line(d1211[0], d1211[1], legend="12-1-1", line_width=2,line_color="black")
p.line(d2411[0], d2411[1], legend="24-1-1", line_width=2,line_color="yellow")


bplt.show(p)

#### Terminology remainder

Processing Time: The time it takes to compute a given batch for all its jobs, end to end. Example: this means a single job which starts at flatMap and ends at saveAsTextFile, and assumes as a prerequisite that the job has been submitted. <br>  <br>
Scheduling Delay: The time taken by Spark Streaming scheduler to submit the jobs of the batch. How is this computed? As we've said, our batch reads from the source every 60 seconds. Now let's assume that a given batch took 80 seconds to compute. This means that we're now 80 - 60 = 20 seconds behind, thus making the scheduling delay 20 seconds long. <br> <br>
Total Delay: This is Scheduling Delay + Processing Time. Following the same example, if we're 20 seconds behind, meaning our scheduling delay is 20 seconds, and the next batch took another 80 seconds to compute, this means that the total delay is now 70 + 40 = 110 seconds long <br> 

### Goal is:

There is no scheduling delay (C), or it's only increasing occasionally and recovers quickly. <br>
The processing time (C) is almost as long as the batch interval (A) but stays below it.  <br>
The median streaming rate (B) is as high as possible without causing trouble for numbers 1 and 2.  <br>